In [25]:
from keras.layers import LSTM, Input, Embedding, Dense, TimeDistributed, Bidirectional, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Nadam
from gensim.models import KeyedVectors
import pickle
import numpy as np

In [47]:
def get_model(sequence_len, dense_shape, word_embeddings_matrix, lstm_dim):
    text_input = Input(shape=(sequence_len,))
    embedding_layer = Embedding(word_embeddings_matrix.shape[0], word_embeddings_matrix.shape[1],
                               trainable=False,weights=[word_embeddings_matrix])
    x = embedding_layer(text_input)
    x = Bidirectional(LSTM(lstm_dim, return_sequences=False))(x)
    x = Dense(dense_shape)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    model_prediction = Dense(6, activation='sigmoid')(x)
    model = Model(inputs=[text_input], outputs=[model_prediction])
    model.compile(optimizer=Nadam(lr=0.001),loss='binary_crossentropy', metrics=['acc'])
    return model

In [8]:
word2vec = KeyedVectors.load_word2vec_format('/Users/freddyalfonsoboulton/wordembeddings/word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [19]:
word_index_dict = pickle.load(open("../data/processed/word_index_dictionary.pkl",'rb'))

In [33]:
def get_embedding_weights(word_index_dict, word2vec):
    embedding_dim = len(word2vec.wv['coffee'])
    embedding_matrix = np.zeros((len(word_index_dict) + 1, embedding_dim))
    for word, i in word_index_dict.items():
        if word in word2vec.wv.vocab:
            embedding_matrix[i] = word2vec.wv[word]
    return embedding_matrix

In [34]:
weights = get_embedding_weights(word_index_dict, word2vec)

In [36]:
np.savez_compressed("../data/processed/embedding_weights.npz",weights = weights)

In [57]:
weights = np.load("../data/processed/embedding_weights.npz")

In [55]:
train_data = np.load("../data/processed/train_data.npz")
val_data = np.load("../data/processed/val_data.npz")

In [48]:
model = get_model(200,15,weights,30)

In [59]:
weights["weights"].shape

(394788, 300)

In [56]:
model.fit(train_data['text'], train_data['targets'], batch_size=28, epochs=1,
          validation_data=(val_data['text'],val_data['targets']))

Train on 143613 samples, validate on 15958 samples
Epoch 1/1
   364/143613 [..............................] - ETA: 5309s - loss: 0.6598 - acc: 0.6378

KeyboardInterrupt: 